In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# 加載已保存的模型
model = load_model('model_new_training_optimal.h5', compile=False)

def emotion_recog(frame):
    emotions = []

    # 防止使用 OpenCL 及不必要的日誌訊息
    cv2.ocl.setUseOpenCL(False)

    # 將每個標籤分配給情緒的字典
    emotion_dict = {0: "Angry", 1: "Happy", 2: "Sad", 3: "Calm"}

    # 加載人臉檢測分類器
    facecasc = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    if facecasc.empty():
        raise IOError('Could not load haarcascade_frontalface_default.xml')

    # 將圖像轉換為灰度圖
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=2)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 255), 3)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        emotion_label = emotion_dict[maxindex]
        cv2.putText(frame, emotion_label, (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        emotions.append(emotion_label)

        plt.figure()
        plt.imshow(cv2.cvtColor(cv2.resize(frame[y:y + h, x:x + w], (48, 48)), cv2.COLOR_BGR2RGB))
        plt.title(emotion_label)
        plt.axis('off')
        plt.show()
        
        # 只處理第一個檢測到的臉部
        break

    return frame, emotions

def display_image(image):
    """使用 matplotlib 顯示圖像。"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('on')
    plt.show()

# 從文件系統讀取圖像
input_image = cv2.imread('sad face.jpg')

# 執行情緒識別
output_image, predicted_emotions = emotion_recog(input_image)
print("predicted output:", predicted_emotions)
final_emotion = predicted_emotions

# 顯示結果
print("原圖")
display_image(output_image)


In [ ]:
# -*- coding: utf-8 -*-
"""Gaussian_Mixture_Models.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1ABJQ3FQEiI5NQFUH5S2dlHSUKBcMvnYp
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('genres_v2.csv')
df.head()

df[df['song_name'].isnull()]

#dropping rows with no song names and no uri and then dropping duplicates as well

# Drop rows with no song names and no URI
df = df.dropna(subset=['song_name', 'uri'])

# Remove duplicate rows based on 'song_name' and 'uri'
df = df.drop_duplicates(subset=['song_name', 'uri'])

# Get the count of final rows
final_row_count = len(df)

print("Count of final rows:", final_row_count)

cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','uri','genre','song_name']
filtered_df = df[cols]
filtered_df

# 加载数据
# 假设 filtered_df 已经定义并包含了数据
num_cols = [i for i in filtered_df.columns if filtered_df[i].dtype != 'object']
scaler = StandardScaler()

# 使用 .loc 方法进行赋值
filtered_df.loc[:, num_cols] = scaler.fit_transform(filtered_df[num_cols])

from sklearn.decomposition import PCA

n_components = 2 # 或者 3 进行3D可视化
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(filtered_df[num_cols])
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)])

n_clusters = 4  # 假设有4个聚类
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
pca_df['cluster'] = gmm.fit_predict(pca_result)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=pca_df['cluster'], palette='viridis', s=50)
plt.title(f'PCA Results with GMM Clustering with {n_clusters} clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# 绘制每个簇的中心点
centers = gmm.means_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, marker='X', edgecolors='black', label='Centers')

plt.legend()
plt.show()

from sklearn.metrics.pairwise import cosine_similarity

# 计算每个簇的均值向量
cluster_means = pca_df.groupby('cluster').mean().values

# 计算不同簇之间的余弦相似度
inter_cluster_cosine_sim = cosine_similarity(cluster_means)

# 打印不同簇之间的余弦相似度矩阵
print("Inter-cluster cosine similarity matrix:")
sns.heatmap(pd.DataFrame(inter_cluster_cosine_sim, index=np.unique(pca_df['cluster']), columns=np.unique(pca_df['cluster'])), cmap='viridis', annot=True, fmt=".2f", annot_kws={"size": 14})

# 可视化不同簇之间的余弦相似度矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(pd.DataFrame(inter_cluster_cosine_sim, index=np.unique(pca_df['cluster']), columns=np.unique(pca_df['cluster'])), cmap='viridis', annot=True)
plt.title('Cosine Similarity Matrix Between Clusters')
plt.show()

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# 假设 filtered_df 已经定义并包含了数据
num_cols = [i for i in filtered_df.columns if filtered_df[i].dtype != 'object']
scaler = StandardScaler()

# 使用 .loc 方法进行赋值
filtered_df.loc[:, num_cols] = scaler.fit_transform(filtered_df[num_cols])

# 进行 PCA
n_components = 3  # 调整主成分数量
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(filtered_df[num_cols])
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)])

# 进行 GMM 分群
n_clusters = 4  # 调整簇数量
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
pca_df['cluster'] = gmm.fit_predict(pca_result)

# 可视化 PCA 3D 结果
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图
scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], c=pca_df['cluster'], cmap='viridis', s=50)

# 添加每个簇的中心点
centers = gmm.means_
ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2], c='red', s=200, marker='X', edgecolors='black', label='Centers')

# 添加图例和标签
legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
ax.add_artist(legend1)
ax.set_title(f'PCA Results with GMM Clustering with {n_clusters} clusters')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')

plt.show()

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import plotly.express as px
import plotly.graph_objects as go

# 假设 filtered_df 已经定义并包含了数据
num_cols = [i for i in filtered_df.columns if filtered_df[i].dtype != 'object']
scaler = StandardScaler()

# 使用 .loc 方法进行赋值
filtered_df.loc[:, num_cols] = scaler.fit_transform(filtered_df[num_cols])

# 进行 PCA
n_components = 3  # 调整主成分数量
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(filtered_df[num_cols])
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)])

# 进行 GMM 分群
n_clusters = 4  # 调整簇数量
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
pca_df['cluster'] = gmm.fit_predict(pca_result)

# 绘制3D PCA结果
fig = px.scatter_3d(
    pca_df, x='PC1', y='PC2', z='PC3',
    color='cluster',
    title='PCA Results with GMM Clustering',
    labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2', 'PC3': 'Principal Component 3'},
    opacity=0.7  # 使点稍微透明，以便更好地观察
)

# 添加每个簇的中心点
centers = gmm.means_
center_df = pd.DataFrame(centers, columns=['PC1', 'PC2', 'PC3'])

# 使用不同的方式添加簇中心点，增强可视化效果
fig.add_trace(go.Scatter3d(
    x=center_df['PC1'], y=center_df['PC2'], z=center_df['PC3'],
    mode='markers',
    marker=dict(size=10, color='red', symbol='x'),
    name='Cluster Centers'
))

# 更新图表布局以增加图的尺寸
fig.update_layout(
    width=1000,  # 图的宽度
    height=800,  # 图的高度
    scene=dict(
        xaxis_title='Principal Component 1',
        yaxis_title='Principal Component 2',
        zaxis_title='Principal Component 3'
    )
)

fig.show()

In [ ]:
import cv2

# 打開攝像頭
cap = cv2.VideoCapture(0)

# 讀取一幀
ret, frame = cap.read()
if not ret:
    print("Failed to capture image")
    cap.release()
    exit()

# 獲取幀的高度和寬度
frame_height, frame_width, _ = frame.shape

# 創建VideoWriter對象，定義視頻編碼和輸出文件名
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

print("Processing Video...")

# 執行情緒識別
output_frame, predicted_emotions = emotion_recog(frame)

# 檢查output_frame的類型和形狀，確保格式正確
print(type(output_frame), output_frame.shape)
print("Predicted Emotions:", predicted_emotions)

if predicted_emotions == ['Happy']:
    final_emotion = 'Happy'
elif predicted_emotions == ['Angry']:
    final_emotion = 'Calm'
elif predicted_emotions == ['Calm']:
    final_emotion = 'Angry'
elif predicted_emotions == ['Sad']:
    final_emotion = 'Sad'

# 寫入幀到文件
out.write(output_frame)

# 釋放資源
out.release()
cap.release()

# 顯示結果
def display_image(image):
    """使用 matplotlib 顯示圖像。"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('on')
    plt.show()

display_image(output_frame)

In [ ]:
# pca_df的結果加上情緒標籤
pca_df['mood'] = np.random.choice(['Sad', 'Calm', 'Angry', 'Happy'], len(pca_df))

# 先存到final_df後再做處理(把song_name放回去、刪掉song_name是NaN的資料)
final_df = pd.concat([df[['song_name', 'uri']], pca_df], axis=1)
# 删除 song_name 是 NaN 的行
final_df = final_df.dropna(subset=['song_name'])



# detected_mood = input("input（Sad, Calm, Angry, Happy）: ")
detected_mood = final_emotion
# print(final_emotion)
valid_moods = ['Sad', 'Calm', 'Angry', 'Happy']


if detected_mood not in valid_moods:
    print("not found please input：Sad or Calm or Angry or Happy")
else:
    filtered_by_mood = final_df[final_df['mood'] == detected_mood]
    if filtered_by_mood.empty:
        print(f"not found any song with {detected_mood} emotion。")
    else:
        from selenium import webdriver
        from selenium.webdriver.common.keys import Keys
        from selenium.webdriver.common.by import By
        from selenium.webdriver.chrome.service import Service
        from selenium.webdriver.chrome.options import Options
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC    
        import time
        import os
        random_songs = filtered_by_mood.sample(3)['song_name']
        print("random song with", detected_mood, "emotion is: ")
        print(random_songs.to_list())

        songs = [{"song_name": song_name} for song_name in random_songs]

        # chrome初始設定
        chrome_options = Options()
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("--disable-extensions")
        chrome_driver_path = os.path.join('chromedriver.exe')


from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome(options=chrome_options)

# try:
from selenium import webdriver
for song in songs:
    song_name = song['song_name']
    driver.get("https://www.youtube.com")
    time.sleep(2) 

    # 搜尋
    search_box = driver.find_element(By.NAME, "search_query")
    search_box.send_keys(song_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2) 
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "video-title")))

    # 選第二個影片(第一個太常是廣告了 可惡)
    search_results = driver.find_elements(By.ID, "video-title")
    second_video = search_results[1]
    second_video.click()

    time.sleep(5) 

    # 影片播的時長
    time.sleep(30) # 30sec

# finally:
    time.sleep(5)
    driver.quit()